In [128]:
import pandas as pd

import numpy as np
import psycopg2
from sqlalchemy import create_engine



In [9]:
bizinfo_ON_df = pd.read_csv('business_info_ontario.csv')
bizinfo_ON_df.head()

/Users/my_macbook/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,10,28,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,address,attributes.AcceptsInsurance,attributes.AgesAllowed,attributes.Alcohol,attributes.Ambience,attributes.BYOB,attributes.BYOBCorkage,attributes.BestNights,attributes.BikeParking,...,hours.Wednesday,is_open,latitude,longitude,name,postal_code,review_count,stars,state,checkin_date
0,0,30 Eglinton Avenue W,NaN,NaN,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,...,9:0-0:0,1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON,"2010-07-05 16:56:31, 2010-12-21 20:30:01, 2010..."
1,1,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9:0-20:0,1,43.599475,-79.711584,The UPS Store,L5V 0B1,3,2.5,ON,"2014-03-31 21:46:17, 2015-01-27 19:05:03, 2018..."
2,2,1455 16th Avenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,...,16:0-23:0,1,43.861503,-79.388499,Hot Yoga Wellness,L4B 3G6,4,4.0,ON,"2016-04-19 17:38:29, 2016-04-22 01:14:54, 2016..."
3,3,"445 King Street W, Suite 101",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6:0-22:0,1,43.645041,-79.395799,Totum Life Science,M5V 1K4,23,4.0,ON,"2010-10-23 14:00:26, 2010-10-24 16:33:33, 2010..."
4,4,"865 York Mills Road, Unit 1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,...,9:0-20:0,1,43.754093,-79.349548,Bakery Gateau,M3B 1Y6,8,4.5,ON,"2016-01-24 00:15:08, 2016-01-25 22:42:18, 2016..."


In [11]:
bizinfo_ON_df.drop(['Unnamed: 0'],inplace=True, axis=1)

In [12]:
bizinfo_ON_df.count()

address                                  33012
attributes.AcceptsInsurance                566
attributes.AgesAllowed                      16
attributes.Alcohol                       11208
attributes.Ambience                      11561
attributes.BYOB                              0
attributes.BYOBCorkage                       1
attributes.BestNights                      533
attributes.BikeParking                   17313
attributes.BusinessAcceptsBitcoin            2
attributes.BusinessAcceptsCreditCards      613
attributes.BusinessParking               21523
attributes.ByAppointmentOnly              5050
attributes.Caters                         9030
attributes.CoatCheck                       648
attributes.Corkage                           0
attributes.DietaryRestrictions              17
attributes.DogsAllowed                    1590
attributes.DriveThru                       552
attributes.GoodForDancing                  855
attributes.GoodForKids                   14726
attributes.Go

In [13]:
tip_df = pd.read_csv('csv_out/tip.csv')
tip_df.head()

,business_id,compliment_count,date,text,user_id
0,VaKXUpmWTTWDKbpJ3aQdMw,0,2014-03-27 03:51:24,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag
1,OPiPeoJiv92rENwbq76orA,0,2013-05-25 06:00:56,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg
2,5KheTjYPu1HcQzQFtm4_vw,0,2011-12-26 01:46:17,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q
3,TkoyGi8J7YFjA6SbaRzrxg,0,2014-03-23 21:32:49,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA
4,AkL6Ous6A1atZejfZXn1Bg,0,2012-10-06 00:19:27,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ


In [18]:
tip_df.columns = ['business_id','tip_compliment_count','tip_date','tip_text','user_id']
tip_df.count()

business_id             1223094
tip_compliment_count    1223094
tip_date                1223094
tip_text                1223090
user_id                 1223094
dtype: int64

In [47]:
ON_business_id_df = bizinfo_ON_df[['business_id']]
ON_business_id_df.count()

business_id    33412
dtype: int64

In [59]:
ON_business_id_df.dropna()
ON_business_id_df.count()

business_id    33412
dtype: int64

In [60]:
ON_biztips_df = pd.merge(ON_business_id_df,tip_df,on='business_id',how='left')
ON_biztips_df.count()

business_id             129212
tip_compliment_count    117518
tip_date                117518
tip_text                117517
user_id                 117518
dtype: int64

In [73]:
# Drop rows with missing values and drop duplicate
ON_biztips_df.dropna(inplace=True)
ON_biztips_df.drop_duplicates(inplace=True)

ON_biztips_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 117500 entries, 0 to 129211
Data columns (total 5 columns):
business_id             117500 non-null object
tip_compliment_count    117500 non-null float64
tip_date                117500 non-null object
tip_text                117500 non-null object
user_id                 117500 non-null object
dtypes: float64(1), object(4)
memory usage: 5.4+ MB


In [88]:
combine = lambda x: ','.join(x)
ON_biztips_df1 = ON_biztips_df.groupby('business_id').agg({'user_id': combine,'tip_text':combine,\
                                                           'tip_compliment_count':'sum','tip_date': combine}).reset_index()


In [89]:
ON_biztips_df1.tail(10)

,business_id,user_id,tip_text,tip_compliment_count,tip_date
21708,zyEnTgWFUydT-Qz_3UK-IA,"mLeMygUVo3j-RgP5XJQi8Q,G-6X-llgA_qAxGxocykHzQ,...",No remote for lining up songs. Touch screen me...,0.0,"2015-06-25 19:12:02,2017-10-09 03:12:30,2015-1..."
21709,zy_NHTqtfSrfTGGPoqy4Mw,"crMMCGbeSwO_hEYoJSTdBA,Mvs4_S_Klo-aiR12keyAYQ,...",Worth a try. One one the few sunny patios left...,0.0,"2010-07-01 22:49:12,2011-12-13 17:55:21,2013-0..."
21710,zyw5DjrRks7a8OhmBsgCQQ,"MfWGWVuUo9bptZ9VcQTC5Q,Win2OfYr2Sg2xFMnr2xwCQ,...","Tankhouse!,great friendly service and decent f...",0.0,"2012-03-16 01:57:57,2012-03-24 16:42:07,2012-0..."
21711,zz-f4Xrs1OGOhybeQaYgFQ,"W6CrBiuPNADGDIH_m39_aw,uylK6FRqzP-E4TmZEX6nwg,...",Quality & indie films in Toronto including ani...,0.0,"2014-08-26 20:26:28,2018-03-08 23:25:34,2010-0..."
21712,zz3CqZhNx2rQ_Yp6zHze-A,"sYl1niCXXzFbyI4UPouM3w,yxHXDnNMcBK8mS4Hyo8BOw,...",#LoserKaraoke with @JasonRolland is now on Tue...,0.0,"2012-03-21 02:17:46,2011-11-02 21:15:32,2011-0..."
21713,zzCsh8Rx40ZXcyDrzFnfOQ,HO3kgFkq7AMmkcE7QvUGLQ,Nephews favourite place for Lamb Curry,0.0,2012-01-08 02:18:38
21714,zzH3E5DA8eMzFwv2k5izyw,XElHLvh6rbyfhUmQq8YHMA,Good meals with fare process.,0.0,2018-07-08 01:41:02
21715,zzMcX99BPGSOFMZ4boINSQ,Wj0tIFlWz_1Uam1VDWYG5g,You can also order ice cream photo cakes here.,0.0,2017-09-04 21:26:56
21716,zzUj3ej4vm_DtvRxNvWDEw,"rGd8YUtvhSdMm8-9cMdQ4Q,s5hemOR2IxcW-0MYA6hGug,...","Half priced wings on Sundays, Mondays, and Tue...",1.0,"2013-06-17 23:03:23,2012-01-15 03:38:13,2016-0..."
21717,zze6IysT7bJFS8gvi6fZ2A,"fHdNrSQhhkCF4Pq1nWnMyA,i_cRb1RgMlhMj-M1Hx_hYw,...",One of my favourite places to come with my lap...,0.0,"2013-09-12 17:36:08,2010-11-04 12:25:00,2015-1..."


In [90]:
bizinfo_ON_tip_df = pd.merge(bizinfo_ON_df,ON_biztips_df1,on='business_id',how='left')

In [91]:
bizinfo_ON_tip_df.head()

,address,attributes.AcceptsInsurance,attributes.AgesAllowed,attributes.Alcohol,attributes.Ambience,attributes.BYOB,attributes.BYOBCorkage,attributes.BestNights,attributes.BikeParking,attributes.BusinessAcceptsBitcoin,...,name,postal_code,review_count,stars,state,checkin_date,user_id,tip_text,tip_compliment_count,tip_date
0,30 Eglinton Avenue W,NaN,NaN,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,NaN,...,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON,"2010-07-05 16:56:31, 2010-12-21 20:30:01, 2010...","fENhF1F6I9c2ssnkfprYzw,RscJR5zqb3WL4eQezBX5rA,...","Spring & egg rolls are great, get them with ex...",1.0,"2016-08-08 01:48:14,2013-06-21 23:44:22,2017-1..."
1,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,The UPS Store,L5V 0B1,3,2.5,ON,"2014-03-31 21:46:17, 2015-01-27 19:05:03, 2018...",jnB_saJqNfOmVoCWquhAzg,Good service,0.0,2014-03-31 21:46:39
2,1455 16th Avenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,Hot Yoga Wellness,L4B 3G6,4,4.0,ON,"2016-04-19 17:38:29, 2016-04-22 01:14:54, 2016...","CxDOIDnH8gp9KXzpBHJYXw,5vRBvQEbR12jXzC50cBYqw","Replacing My Yoga Studio,Went there for the fi...",0.0,"2016-04-19 16:16:40,2017-09-20 15:10:00"
3,"445 King Street W, Suite 101",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Totum Life Science,M5V 1K4,23,4.0,ON,"2010-10-23 14:00:26, 2010-10-24 16:33:33, 2010...","n1bfatw5wT1eRYWmHSLo9g,gLSQCEhGy7-uRnNbNnVykg","One-week guest pass is $44.,Very average. Best...",0.0,"2014-08-07 03:35:10,2013-07-28 21:31:25"
4,"865 York Mills Road, Unit 1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,Bakery Gateau,M3B 1Y6,8,4.5,ON,"2016-01-24 00:15:08, 2016-01-25 22:42:18, 2016...","keLUgL_4y60BkppiAsIk8Q,keLUgL_4y60BkppiAsIk8Q",Tried the tiramisu mini cake $16.. very light!...,1.0,"2016-03-22 20:21:20,2016-01-23 20:02:08"


In [115]:
bizinfo_ON_tip_df.count()

address                                  33012
attributes.AcceptsInsurance                566
attributes.AgesAllowed                      16
attributes.Alcohol                       11208
attributes.Ambience                      11561
attributes.BYOB                              0
attributes.BYOBCorkage                       1
attributes.BestNights                      533
attributes.BikeParking                   17313
attributes.BusinessAcceptsBitcoin            2
attributes.BusinessAcceptsCreditCards      613
attributes.BusinessParking               21523
attributes.ByAppointmentOnly              5050
attributes.Caters                         9030
attributes.CoatCheck                       648
attributes.Corkage                           0
attributes.DietaryRestrictions              17
attributes.DogsAllowed                    1590
attributes.DriveThru                       552
attributes.GoodForDancing                  855
attributes.GoodForKids                   14726
attributes.Go

In [117]:
photo_df = pd.read_csv('csv_out/photo.csv')
photo_df.head()

,business_id,caption,label,photo_id
0,rcaPajgKOJC2vo_l3xa42A,NaN,inside,MllA1nNpcp1kDteVg6OGUw
1,Kn23LDd740SBVJ7mum0fwg,NaN,inside,YjxBE88Bf6CmTEF2LP1UNA
2,ZkGDCVKSdf8m76cnnalL-A,NaN,food,1f7izSjM0WjkDRIVbPy1yw
3,bF8gv7k_rwZtiDLP2ZB04w,NaN,inside,NcSlcDTEEeOaixotOPk-rA
4,50Anorn0DJXFhBr9a9_gHQ,NaN,inside,5IiIo5UKEW0lWqZ6sWrY_A


In [118]:
ON_biz_photo_df = pd.merge(ON_business_id_df,photo_df,on='business_id',how='left')
ON_biz_photo_df.count()






business_id    68035
caption        18134
label          42054
photo_id       42054
dtype: int64

In [119]:
ON_biz_photo_df.drop_duplicates(inplace=True)
ON_biz_photo_df.fillna('null',inplace=True)
ON_biz_photo_df.count()


business_id    68016
caption        68016
label          68016
photo_id       68016
dtype: int64

In [120]:
ON_biz_photo_df = ON_biz_photo_df.groupby('business_id').agg({'photo_id':combine,'label':combine,'caption':combine}).reset_index()
ON_biz_photo_df.count()

business_id    33412
photo_id       33412
label          33412
caption        33412
dtype: int64

In [121]:



Ontario_bizinfo_df = pd.merge(bizinfo_ON_tip_df,ON_biz_photo_df,on='business_id',how='left')

In [122]:
Ontario_bizinfo_df.count()

address                                  33012
attributes.AcceptsInsurance                566
attributes.AgesAllowed                      16
attributes.Alcohol                       11208
attributes.Ambience                      11561
attributes.BYOB                              0
attributes.BYOBCorkage                       1
attributes.BestNights                      533
attributes.BikeParking                   17313
attributes.BusinessAcceptsBitcoin            2
attributes.BusinessAcceptsCreditCards      613
attributes.BusinessParking               21523
attributes.ByAppointmentOnly              5050
attributes.Caters                         9030
attributes.CoatCheck                       648
attributes.Corkage                           0
attributes.DietaryRestrictions              17
attributes.DogsAllowed                    1590
attributes.DriveThru                       552
attributes.GoodForDancing                  855
attributes.GoodForKids                   14726
attributes.Go

In [124]:
Ontario_bizinfo_df.tail(10)

,address,attributes.AcceptsInsurance,attributes.AgesAllowed,attributes.Alcohol,attributes.Ambience,attributes.BYOB,attributes.BYOBCorkage,attributes.BestNights,attributes.BikeParking,attributes.BusinessAcceptsBitcoin,...,stars,state,checkin_date,user_id,tip_text,tip_compliment_count,tip_date,photo_id,label,caption
33402,"221 Davenport Rd Toronto, ON M5R IJ3, Unit 9",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,ON,NaN,NaN,NaN,NaN,NaN,null,null,null
33403,"30 Broadleaf Avenue, Unit 5",NaN,NaN,NaN,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,NaN,...,3.0,ON,"2013-06-02 23:04:51, 2014-07-24 19:17:08, 2015...","2uoQbePwGv9RupGchFvrWg,AQuRcbUgHt3KZlvzyHCeag",Beside a school so be aware of the time of day...,0.0,"2014-02-07 01:41:11,2013-06-02 23:05:21",null,null,null
33404,"9275 Hwy 48, Unit 11",NaN,NaN,'none',"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN,False,NaN,...,3.0,ON,"2013-03-30 22:04:45, 2014-08-11 01:11:12, 2016...","9GU6jb_nmq-cUcpqbfk_Fw,q8c55yrIOBGZJMlTuVXDlw,...",Kings kitchen Sea Treasurer is a great dish. J...,0.0,"2010-08-14 12:49:54,2010-10-21 03:56:20,2018-0...",null,null,null
33405,2046 Sheppard Avenue E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,3.5,ON,"2011-02-23 13:37:47, 2012-11-23 00:28:37, 2013...","pn_flI3EBNugBEYFp9okxQ,pn_flI3EBNugBEYFp9okxQ,...",They no longer serve regular coffee nor give a...,0.0,"2016-09-28 01:19:40,2016-09-28 01:11:33,2012-1...",null,null,null
33406,81 Yorkville Avenue,NaN,NaN,u'full_bar',"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN,True,NaN,...,3.5,ON,"2010-02-15 02:38:02, 2010-03-10 23:21:17, 2010...","_MfYDCxHmMIQtrIIUBPW5Q,m_a6wonkZYJbcvi6cufsXA,...","Everything is amazing!,Add an egg to the bibim...",0.0,"2010-02-15 02:38:02,2016-02-02 17:21:30,2011-0...",UXI24IWkufc3qj2dOaJ9WQ,food,Bulgogi bibimbap
33407,8920 Highway 50,NaN,NaN,u'none',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,NaN,...,3.0,ON,"2014-05-14 18:04:07, 2014-06-23 14:58:40, 2014...","7qlxzG8zSM7hVMiSNNKrnA,1s1JFYzVBh9ZbKRdAgxNTQ,...",Fantastic food and great service. These guys m...,0.0,"2016-04-28 20:24:08,2014-09-26 17:01:52,2015-0...",GtVZVnZCsec4vAjFcy23aQ,outside,null
33408,578 Yonge Street,NaN,NaN,u'beer_and_wine',"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN,True,NaN,...,4.0,ON,"2016-04-18 01:45:13, 2016-04-18 16:13:30, 2016...","0IwZb1ePNpSVYcCdMtGoQw,n2mAOAYORrenCeFlgRphjA,...",The best time to come here is during lunch tim...,0.0,"2017-03-28 00:51:31,2016-05-09 19:39:28,2016-0...","wghfoIrMYCFienWusW-iNg,4SNf37XZJb5wz6DksX8qLA,...","food,food,food,inside,inside,food","null,House pai Thai,null,I love the aesthetics..."
33409,540 Eglinton Avenue W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.5,ON,"2013-01-14 17:30:36, 2017-12-18 16:53:50",NaN,NaN,NaN,NaN,null,null,null
33410,1450 Kingston Rd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,4.5,ON,NaN,NaN,NaN,NaN,NaN,null,null,null
33411,17440 Yonge Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,4.5,ON,"2011-09-25 16:46:23, 2012-10-13 14:00:18, 2012...",mZ1gXzL6Tn5Oky8_j0Kp7g,Lots of great sale accessories and housewares!,0.0,2014-07-13 16:57:29,null,null,null


In [125]:
Ontario_bizinfo_df.to_csv('ontaro_biz_data.csv')

In [127]:
col_list = Ontario_bizinfo_df.columns
col_list

Index(['address', 'attributes.AcceptsInsurance', 'attributes.AgesAllowed',
       'attributes.Alcohol', 'attributes.Ambience', 'attributes.BYOB',
       'attributes.BYOBCorkage', 'attributes.BestNights',
       'attributes.BikeParking', 'attributes.BusinessAcceptsBitcoin',
       'attributes.BusinessAcceptsCreditCards', 'attributes.BusinessParking',
       'attributes.ByAppointmentOnly', 'attributes.Caters',
       'attributes.CoatCheck', 'attributes.Corkage',
       'attributes.DietaryRestrictions', 'attributes.DogsAllowed',
       'attributes.DriveThru', 'attributes.GoodForDancing',
       'attributes.GoodForKids', 'attributes.GoodForMeal',
       'attributes.HairSpecializesIn', 'attributes.HappyHour',
       'attributes.HasTV', 'attributes.Music', 'attributes.NoiseLevel',
       'attributes.Open24Hours', 'attributes.OutdoorSeating',
       'attributes.RestaurantsAttire', 'attributes.RestaurantsCounterService',
       'attributes.RestaurantsDelivery', 'attributes.RestaurantsGoodForGr

In [129]:
#Load into the PostgreSQL
engine = create_engine('postgresql+psycopg2://postgres:postgres@@127.0.0.1/yelp_data')



In [130]:
Ontario_bizinfo_df.to_sql('yelp_on_full_data',con=engine)